In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [12]:
df = pd.read_csv('datos_v2.csv', delimiter=';')
# df2 = pd.read_csv('datos.csv', delimiter=';')

In [11]:
# Unir los dos Dataframe del 20323 y 2024 
# ESTE PASO EL OPCIONAL Solo si cargan los datos df2
# columnasEnComun = ['Temperatura de módulo | Sensor Card / Box (1)','Irradiación | Sensor Card / Box (1)','Temperatura ambiente | Sensor Card / Box (1)','Fecha y hora']
# df = pd.concat([df[columnasEnComun], df2[columnasEnComun]])
# df

,Temperatura de módulo | Sensor Card / Box (1),Irradiación | Sensor Card / Box (1),Temperatura ambiente | Sensor Card / Box (1),Fecha y hora
0,"31,00","288,00","24,00",01.03.2024 15:45
1,"34,00","293,62","24,00",01.03.2024 15:50
2,"34,00","340,00","24,00",01.03.2024 15:55
3,"36,00","469,00","24,00",01.03.2024 16:00
4,"36,00","385,00","24,00",01.03.2024 16:05
...,...,...,...,...
57881,NaN,NaN,NaN,NaN
57882,NaN,NaN,NaN,NaN
57883,NaN,NaN,NaN,NaN
57884,NaN,NaN,NaN,NaN


In [13]:
# Renombrar columnas
df.rename(columns={'Temperatura de módulo | Sensor Card / Box (1)': 'temperaturaSensor'}, inplace=True)
df.rename(columns={'Temperatura ambiente | Sensor Card / Box (1)': 'temperaturaAmbiente'}, inplace=True)
df.rename(columns={'Irradiación | Sensor Card / Box (1)': 'irradiacion'}, inplace=True)
df.rename(columns={'Fecha y hora': 'fecha'}, inplace=True)

# Quedarme con dos columnas de entrada y una de salida(irradiacion)
df = df[['fecha','temperaturaAmbiente','temperaturaSensor','irradiacion']]

# Eliminar los valores nulos
df.dropna(inplace=True)

df['irradiacion'] = df['irradiacion'].apply(lambda x: float(str(x).strip().replace(',','.')))

# Solo quedarme con los valores que hay irradiación (Día)
df = df[df['irradiacion'] > 0]
df


,fecha,temperaturaAmbiente,temperaturaSensor,irradiacion
0,01.03.2024 15:45,"24,00","31,00",288.00
1,01.03.2024 15:50,"24,00","34,00",293.62
2,01.03.2024 15:55,"24,00","34,00",340.00
3,01.03.2024 16:00,"24,00","36,00",469.00
4,01.03.2024 16:05,"24,00","36,00",385.00
...,...,...,...,...
12395,26.04.2024 09:20,"23,00","33,00",302.00
12396,26.04.2024 09:30,"23,00","34,00",303.00
12397,26.04.2024 09:40,"23,00","32,99",264.03
12398,26.04.2024 09:50,"23,00","32,00",268.00


In [14]:
# Transformar la fecha a datetime
df['fecha'] = pd.to_datetime(df['fecha'], format='%d.%m.%Y %H:%M')
df['hora'] = [i.hour for i in df['fecha']]
df['minuto'] = [i.minute for i in df['fecha']]
# Parsear las columnas temperaturas a float
df['temperaturaAmbiente'] = df['temperaturaAmbiente'].apply(lambda x: float(str(x).strip().replace(',','.')))
df['temperaturaSensor'] = df['temperaturaSensor'].apply(lambda x: float(str(x).strip().replace(',','.')))

### Separar los datos para Entrenamiento y TEST

In [15]:
X = df[['temperaturaAmbiente', 'hora', 'minuto']].values

# La radiación será nuestra variable de salida
y = df['irradiacion'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir la arquitectura de la RNA
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(3,)), #  variables de entrada
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Capa de salida con una sola neurona para predecir la radiación
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)



Epoch 1/50
126/126 [==============================] - 1s 3ms/step - loss: 137278.4688 - val_loss: 84005.0078
Epoch 2/50
126/126 [==============================] - 0s 2ms/step - loss: 72425.7969 - val_loss: 68455.7891
Epoch 3/50
126/126 [==============================] - 0s 2ms/step - loss: 61550.8945 - val_loss: 59846.5273
Epoch 4/50
126/126 [==============================] - 0s 2ms/step - loss: 55518.7734 - val_loss: 54441.0625
Epoch 5/50
126/126 [==============================] - 0s 2ms/step - loss: 51004.0312 - val_loss: 50445.2891
Epoch 6/50
126/126 [==============================] - 0s 2ms/step - loss: 47112.2422 - val_loss: 47167.0352
Epoch 7/50
126/126 [==============================] - 0s 2ms/step - loss: 44418.9570 - val_loss: 44867.4062
Epoch 8/50
126/126 [==============================] - 0s 2ms/step - loss: 42495.0664 - val_loss: 43529.1211
Epoch 9/50
126/126 [==============================] - 0s 2ms/step - loss: 40985.1719 - val_loss: 42070.9844
Epoch 10/50
126/126 [======

In [16]:
# Evaluar el modelo
loss = model.evaluate(X_test, y_test)
print("Loss en el conjunto de prueba:", loss)

40/40 [==============================] - 0s 2ms/step - loss: 20084.0703
Loss en el conjunto de prueba: 20084.0703125


In [17]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Coeficiente de determinación (R²):", r2)

40/40 [==============================] - 0s 1ms/step
Coeficiente de determinación (R²): 0.7516623196755863


In [18]:
# Guardar modelo
model.save('modelo_rna_v3')

INFO:tensorflow:Assets written to: modelo_rna_v3\assets


INFO:tensorflow:Assets written to: modelo_rna_v3\assets


In [19]:
# Cargar modelo
from tensorflow.keras.models import load_model

# Cargar el modelo
modelo_cargado = load_model('modelo_rna_v3')

# Confirmación de que el modelo se ha cargado correctamente
print("Modelo cargado")
modelo_cargado.predict([[23,9,40]])

Modelo cargado
1/1 [==============================] - 0s 70ms/step


array([[451.38565]], dtype=float32)